In [24]:
import os
from dapper.mods import Chronology

from utils import setup as setup_lorenz
from utils import simulate_ens, NNPredictor, SetupBuilder, plot_L96_2D
import numpy as np
import matplotlib.pyplot as plt

In [33]:
#################
# General Setup #
#################

N = 100 # Ensemble size
datadir = 'example_data'  # Directory where to save the results
weights1 = 'weights_init.h5'
weights2 = 'weights_nn.h5'

m = 40  # size of the state space
p = 20  # Number of obs at each time step (50%)
#std_m = 0.1  # standard deviation of model noise
std_o = 1.  # standard devation of observational noise
Texpe = 5. # Length of the experiment in model time unit


In [14]:
######################
# Initial simulation #
######################

# Init simulation:
xx_spinup = simulate_ens(setup_lorenz)

# initial state (discard the first 100 time steps as spinup):
xx_init = xx_spinup[100:]

Simul:   0%|          | 0/20480 [00:00<?, ?it/s]

In [15]:
xx_init.shape

(20381, 40)

In [16]:
##################
# reference run  #
##################

# setup builder of the run:
sb = SetupBuilder(t=Chronology(0.05, dkObs=1, T=Texpe, BurnIn=1),
	p=p,
	std_o=std_o,
	data=xx_init)

# Define the setup run for the true simulation:
setup_true = sb.setup()

In [34]:
##########################
# Machine learning setup #
##########################
# Parameters of the neural network (architecture + training):
param_nn = {'archi': ((24, 5, 'relu', 0.0), (37, 5, 'relu', 0.0)),  # CNN layer setup
	'bilin': True,  # activate bilinear layer
	'batchnorm': True,  # activate batchnorm normalization
	'reg': ('ridge', 1e-4),  # L2 regularization for the last layer
	'weighted': True,  # Using a variance matrix for the loss function
	'finetuning': False,  # Deactivate a finetuning of the last layer after optimization
	'npred': 1,  # Number of forecast time step in the loss function
	'Nepochs': 1,  # Number of epochs
	'batch_size': 256  # Batchsize during the training
}
nn1 = NNPredictor(m, **param_nn)

# Load the neural net with  weights
nn1._smodel.load_weights(os.path.join(datadir, weights1))

nn2 = NNPredictor(m, **param_nn)

# Load the neural net with  weights
nn2._smodel.load_weights(os.path.join(datadir, weights2))


In [35]:
# Define surrogate model setup:
setup1 = nn1.define_setup(setup_true)
setup2 = nn2.define_setup(setup_true)

# simulation
xsim_true      = simulate_ens(setup_true, N=N)
x0 = xsim_true[0]

Simul:   0%|          | 0/100 [00:00<?, ?it/s]

In [36]:
########
# plot #
########

xsim_surrogate1 = simulate_ens(setup1, N=N, Xinit=x0)

# plot
fig = plot_L96_2D(xsim_true[:,0], xsim_surrogate1[:,0], 1.67*setup_true.t.tt, labels=['True','Surrogate'])
fig.savefig(os.path.join(datadir, 'simulation_init.png'))


Simul:   0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
########
# plot #
########

xsim_surrogate2 = simulate_ens(setup2, N=N, Xinit=x0)

# plot
fig = plot_L96_2D(xsim_true[:,0], xsim_surrogate2[:,0], 1.67*setup_true.t.tt, labels=['True','Surrogate'])
fig.savefig(os.path.join(datadir, 'simulation_final.png'))

Simul:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
########
# plot #
########
# Define surrogate model setup:
setup = nn.define_setup(setup_true)

# Init at the end of the training period:


# simulation
xsim_true      = simulate_ens(setup_true, N=N)
x0 = xsim_true[0]
xsim_surrogate = simulate_ens(setup, N=N, Xinit=x0)

# plot
fig = plot_L96_2D(xsim_true[:,0], xsim_surrogate[:,0], 1.67*setup_true.t.tt, labels=['True','Surrogate'])
fig.savefig(os.path.join(datadir, 'simulation_init.png'))

In [47]:
rmse1 = np.sqrt(np.mean(np.square(xsim_true-xsim_surrogate1),axis=(1,2))/2)
rmse2 = np.sqrt(np.mean(np.square(xsim_true-xsim_surrogate2),axis=(1,2))/2)
sigma = np.std(xsim_true)

In [49]:
fig, ax = plt.subplots()
ax.plot(1.67*setup_true.t.tt,rmse1, label='hybrid after 1 cycle')
ax.plot(1.67*setup_true.t.tt,rmse2, label='hybrid after 40 cycles')
ax.plot(1.67*setup_true.t.tt,sigma*np.ones_like(setup_true.t.tt), ':k', label='Model standard deviation')

ax.legend()
ax.set_xlabel('lead time (in Lyapunov time)')
ax.set_ylabel('RMSE')
fig.savefig(os.path.join(datadir, 'rmse.png'))